In [1]:
%load_ext lab_black

In [2]:
from layer_native import MulLayer
from add_layer import AddLayer

In [3]:
apple = 100
apple_num = 2
tax = 1.1

In [4]:
# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

In [5]:
# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [6]:
# backword
dprice = 1
dapple_price, dtax = mul_tax_layer.backword(dprice)
dapple, dapple_num = mul_apple_layer.backword(dapple_price)
print(dapple)
print(dapple_num)
print(dtax)

2.2
110.00000000000001
200


In [7]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

In [8]:
# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
mul_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

In [9]:
# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = mul_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

In [10]:
# backword
dprice = 1
dall_price, dtax = mul_tax_layer.backword(dprice)
dapple_price, dorange_price = mul_apple_orange_layer.backword(dall_price)
dapple, dapple_num = mul_apple_layer.backword(dapple_price)
dorange, dorange_num = mul_orange_layer.backword(dorange_price)
print(price)
print(dapple_num)
print(dapple)
print(dorange_num)
print(dorange)
print(dtax)

715.0000000000001
110.00000000000001
2.2
165.0
3.3000000000000003
650


# Gradient Check 勾配確認

In [11]:
import numpy as np
from mnist import load_mnist
from two_layer_net import TwoLayerNet

In [12]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

In [13]:
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

In [14]:
x_batch = x_train[:3]
t_batch = t_train[:3]

In [15]:
grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

In [16]:
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:9.834917302041e-05
b1:0.000754159502158356
W2:0.003242938091271196
b2:0.06666666678634872


# 誤差逆伝播法を使った学習

In [17]:
# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 勾配
    # grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)

    # 更新
    for key in ("W1", "b1", "W2", "b2"):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.08976666666666666 0.091
0.09863333333333334 0.0958
0.0993 0.1032
0.09751666666666667 0.0974


/app/ch5/softmax_with_loss.py:6: RuntimeWarning: invalid value encountered in subtract
  exp_a = np.exp(a - c)  # オーバーフロー対策


0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
